In [81]:
import pandas as pd
import numpy as np
import os

In [82]:
# Data read
file_list = []
for file in os.listdir('../data/data_보통주/'):
    file_list.append(file)

    stock_info = pd.read_csv('../data/data_보통주/{0}'.format(file))

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [83]:
# 빈 로우 채우기
stock_info = pd.read_csv('../data/test_fillrow/000050_경방.csv')

In [115]:
samsung = pd.read_csv('../data/data_보통주/005930_삼성전자.csv')

In [116]:
stock_info = stock_info.set_index(['Unnamed: 0', '시간'])
samsung = samsung.set_index(['Unnamed: 0', '시간'])

In [117]:
samsung = samsung.notnull().replace(True, np.NaN)

In [120]:
samsung.to_csv('../data/fillrow_template.csv', encoding='utf-8 sig')

In [87]:
stock_info = pd.merge(samsung, stock_info, how='left', left_index=True, right_index=True)

In [88]:
stock_info = stock_info.drop(columns=stock_info.columns[0:23], axis=1)

columns_temp = stock_info.columns

bin_columns = []
for column in columns_temp:
    bin_columns.append(column.split('_')[0])

stock_info.columns = bin_columns

In [89]:
stock_info = stock_info.reset_index()

In [97]:
stock_info.bfill()

,Unnamed: 0,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
0,20221103,1530,11000.0,11000.0,11000.0,11000.0,0.0,93.0,1020000.0,4620.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20221103,1520,11000.0,11000.0,11000.0,11000.0,0.0,20.0,220000.0,4620.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20221103,1519,11000.0,11000.0,11000.0,11000.0,0.0,100.0,1100000.0,4620.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20221103,1518,10950.0,10950.0,10950.0,10950.0,0.0,3.0,30000.0,4620.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20221103,1517,10950.0,10950.0,10950.0,10950.0,0.0,3.0,30000.0,4620.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189994,20201028,905,10800.0,10800.0,10800.0,10800.0,0.0,12.0,130000.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189995,20201028,904,10800.0,10800.0,10800.0,10800.0,0.0,12.0,130000.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189996,20201028,903,10900.0,10900.0,10900.0,10900.0,0.0,25.0,270000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189997,20201028,902,10900.0,10900.0,10900.0,10900.0,0.0,25.0,270000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
stock_info[['거래량','거래대금']] = stock_info[['거래량','거래대금']].fillna(0)

In [113]:
stock_info = stock_info.bfill()

In [114]:
stock_info

,Unnamed: 0,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
0,20221103,1530,11000.0,11000.0,11000.0,11000.0,0.0,93.0,1020000.0,4620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20221103,1520,11000.0,11000.0,11000.0,11000.0,0.0,20.0,220000.0,4620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20221103,1519,11000.0,11000.0,11000.0,11000.0,0.0,100.0,1100000.0,4620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20221103,1518,10950.0,10950.0,10950.0,10950.0,0.0,0.0,0.0,4620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20221103,1517,10950.0,10950.0,10950.0,10950.0,0.0,0.0,0.0,4620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189994,20201028,905,10800.0,10800.0,10800.0,10800.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189995,20201028,904,10800.0,10800.0,10800.0,10800.0,0.0,12.0,130000.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189996,20201028,903,10900.0,10900.0,10900.0,10900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189997,20201028,902,10900.0,10900.0,10900.0,10900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
test = pd.read_csv('../data/fillrow_template.csv', encoding='utf-8 sig')
test = test.set_index(['Unnamed: 0', '시간'])
test

시가  고가  저가  종가  전일대비  거래량  거래대금  누적체결매도수량  상장주식수  시가총액  외국인주문한도수량  외국인주문가능수량  외국인현보유수량  외국인현보유비율  수정주가일자  수정주가비율  기관순매수량  기관누적순매수량  등락주선  등락비율  예탁금  주식회전율  거래성립률
Unnamed: 0 시간                                                                                                                                                                     
20221103   1530 NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           1520 NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           1519 NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           1518 NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           1517 NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
...              ..  ..  ..  ..   ...  ...   ...       ...    ...   ...        ...        ...       ...       ...     ...     ...     ...       ...   ...   ...  ...    ...    ...
20201028   905  NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           904  NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           903  NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           902  NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN
           901  NaN NaN NaN NaN   NaN  NaN   NaN       NaN    NaN   NaN        NaN        NaN       NaN       NaN     NaN     NaN     NaN       NaN   NaN   NaN  NaN    NaN    NaN

[189999 rows x 23 columns]